In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [18]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils

In [404]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [6]:
a = np.array([[1]*39])
b = [500]

In [516]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='relu', input_shape=(10,)))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [517]:
model = build_model()

In [324]:
try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

In [466]:
!ls 'rec'

'[2zzz]1556610247.0caoredjiba.txt'


In [24]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [25]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [26]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [511]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l('rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            # does not include the board
            break
            board.append(s['board'][cord])
        # color 
        # board.append(color)
        # utility = [heuristic diff, my pieces difference, danger pieces]
        board.append(s['turns'])
        board += s['utility']
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.95
    curr_decay = 0.95
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    post_value = np.array(value)
    
    # normalize the value
    # post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        # print(post_value)
        pass
    
    return train, value

In [524]:
t,v = PG_log('[2zzz]1556634121.0caoredjiba.txt', value_check=True)
print(v)

[0, 0, -4, -10, 0, 0, -10, -4, -4, -4, -4, 0, 0, 0, -4, -10, -10, -10, 0, 0, 0, 0, 0, -4, -320]
[-0.6664208385899949, -2.2823489345161083, -7.425729418084809, -10.58848292953113, -1.7279777993345462, -4.820211035693861, -12.773724706239838, -6.98294239997695, -7.134172028282967, -7.121068209485777, -6.73673060551905, -5.611792121224909, -10.931876236161605, -20.230724818052646, -35.56737260200673, -52.7232755042004, -67.78788658107362, -87.10597964067613, -110.41366410183092, -150.20389356758312, -194.11681819517963, -238.32452874999996, -277.96999999999997, -308.0, -320]
[-0.6664208385899949, -2.2823489345161083, -7.425729418084809, -10.58848292953113, -1.7279777993345462, -4.820211035693861, -12.773724706239838, -6.98294239997695, -7.134172028282967, -7.121068209485777, -6.73673060551905, -5.611792121224909, -10.931876236161605, -20.230724818052646, -35.56737260200673, -52.7232755042004, -67.78788658107362, -87.10597964067613, -110.41366410183092, -150.20389356758312, -194.1168181951

In [528]:
!ls rec

In [331]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')

In [384]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [351]:
def train(save=True, epochs):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            #print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    if save:
        print("saved")
        save_model()   

In [352]:
train()

Epoch 1/2
323/323 [==============================] - 2s 6ms/step - loss: 0.7966
Epoch 2/2
323/323 [==============================] - 1s 3ms/step - loss: 0.5005
saved


In [518]:
save_model()

In [321]:
del model

In [468]:
model.predict(np.array([[1]*8,[0.5]*8]))

array([[1.6910833 ],
       [0.84554166]], dtype=float32)

In [52]:
def clean_log(logs):
    !rm '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/' *

In [53]:
clean_log(logfs)

rm: cannot remove '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/': Is a directory
rm: cannot remove '__pycache__': Is a directory


In [110]:
os.getcwd()

'/Users/xinyaoniu/Documents/COMP30024-AI/Project2'

In [146]:
!rm -v ./rec/*

removed './rec/[1win]1556549315.0caoredjiba.txt'
removed './rec/[1win]1556549321.0caoredjiba.txt'
removed './rec/[2zzz]1556549305.0caoredjiba.txt'
removed './rec/[2zzz]1556549307.0caoredjiba.txt'
removed './rec/[2zzz]1556549308.0caoredjiba.txt'
removed './rec/[2zzz]1556549309.0caoredjiba.txt'
removed './rec/[2zzz]1556549311.0caoredjiba.txt'


In [353]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee VanGame HardCode HardCode -l logtry -v 0
    if (len(bash_call(['ls', 'rec']))) > 0:
        train(epochs=epochs)
        !rm -v ./rec/*

In [527]:
for i in range(200):
    print("********** starting round", i, "**********")
    full_train_cycle(1, epochs=2)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
22/22 [==============================] - 0s 2ms/step - loss: 14706.7556
Epoch 2/2
22/22 [==============================] - 0s 2ms/step - loss: 14391.4317
saved
removed './rec/[2zzz]1556635787.0caoredjiba.txt'
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
25/25 [==============================] - 0s 2ms/step - loss: 13088.5655
Epoch 2/2
25/25 [==============================] - 0s 2ms/step - loss: 12970.4310
saved
removed './rec/[2zzz]1556635789.0caoredjiba.txt'
********** starting round 2 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
60/60 [==============================] - 0s 2ms/step - loss: 2205.6635
Epoch 2/2
60/60 [==============================] - 0s 2ms/step - loss: 1468.4038
saved
removed './rec/[1win]1

********** starting round 24 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
25/25 [==============================] - 0s 2ms/step - loss: 14191.5466
Epoch 2/2
25/25 [==============================] - 0s 2ms/step - loss: 13936.3554
saved
removed './rec/[2zzz]1556635832.0caoredjiba.txt'
********** starting round 25 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
28/28 [==============================] - 0s 2ms/step - loss: 4561.7734
Epoch 2/2
28/28 [==============================] - 0s 2ms/step - loss: 4322.0779
saved
removed './rec/[2zzz]1556635834.0caoredjiba.txt'
********** starting round 26 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
29/29 [==============================] - 0s 2ms/step - loss: 10829.9571
Epoch 2/2
29/29 [==============================] - 0s 2ms/step - loss: 9296.4695
saved
removed './rec/[2zz

********** starting round 48 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
31/31 [==============================] - 0s 2ms/step - loss: 9481.9748
Epoch 2/2
31/31 [==============================] - 0s 2ms/step - loss: 7917.7942
saved
removed './rec/[2zzz]1556635882.0caoredjiba.txt'
********** starting round 49 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
62/62 [==============================] - 0s 2ms/step - loss: 31367.3386
Epoch 2/2
62/62 [==============================] - 0s 2ms/step - loss: 7497.3546
saved
removed './rec/[1win]1556635884.0caoredjiba.txt'
********** starting round 50 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
28/28 [==============================] - 0s 2ms/step - loss: 14327.8176
Epoch 2/2
28/28 [==============================] - 0s 2ms/step - loss: 14103.3148
saved
removed './rec/[2zzz]

saved
removed './rec/[2zzz]1556635921.0caoredjiba.txt'
********** starting round 72 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
28/28 [==============================] - 0s 2ms/step - loss: 3530.9581
Epoch 2/2
28/28 [==============================] - 0s 1ms/step - loss: 3496.7788
saved
removed './rec/[2zzz]1556635922.0caoredjiba.txt'
********** starting round 73 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
28/28 [==============================] - 0s 2ms/step - loss: 10833.8982
Epoch 2/2
28/28 [==============================] - 0s 2ms/step - loss: 10758.8024
saved
removed './rec/[2zzz]1556635923.0caoredjiba.txt'
********** starting round 74 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
35/35 [==============================] - 0s 2ms/step - loss: 4060.1125
Epoch 2/2
35/35 [==============================] - 0s

23/23 [==============================] - 0s 2ms/step - loss: 15142.9173
saved
removed './rec/[2zzz]1556635961.0caoredjiba.txt'
********** starting round 96 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
25/25 [==============================] - 0s 2ms/step - loss: 13152.2432
Epoch 2/2
25/25 [==============================] - 0s 2ms/step - loss: 12950.7874
saved
removed './rec/[2zzz]1556635963.0caoredjiba.txt'
********** starting round 97 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
22/22 [==============================] - 0s 2ms/step - loss: 14342.5101
Epoch 2/2
22/22 [==============================] - 0s 2ms/step - loss: 14134.0299
saved
removed './rec/[2zzz]1556635965.0caoredjiba.txt'
********** starting round 98 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
32/32 [==============================] - 0s 2ms/st

ValueError: Error when checking input: expected dense_79_input to have shape (10,) but got array with shape (1,)

# CNN attamp

In [226]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [227]:
for i in md.get_weights():
    print(i.shape)

(3, 3, 1, 1)


In [228]:
md.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 1)         9         
_________________________________________________________________
flatten_9 (Flatten)          (None, 676)               0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [229]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.

In [244]:
asdf = md.get_weights()[0].reshape(3,3)

In [230]:
wds = md.get_weights()

In [231]:
convp = wds[0]

In [232]:
convp.shape

(3, 3, 1, 1)

In [233]:
c = convp.reshape(1, 3, 3, 1)

In [234]:
d = c[0].reshape(3,3)

In [235]:
import scipy.signal as signal

In [236]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [245]:
e = signal.convolve2d(a, asdf, mode='valid')

In [246]:
e

array([[-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
      

In [206]:
signal.convolve(e, )

ValueError: volume and kernel should have the same dimensionality